1.导入数据集

In [1]:
from NLP import comment_analysis as ca
from gensim import corpora, models


doc = ca.prepare_news_to_corpora("news540")
doc_tokens = doc["doc_bodies"]
dictionary = corpora.Dictionary(doc_tokens) 

corpus = [dictionary.doc2bow(text) for text in doc_tokens]
tags = doc["tags"]

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.301 seconds.
DEBUG:jieba:Loading model cost 0.301 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


2.导入需要的模型库

In [69]:
from time import time
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
#from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn import metrics

3.异常数据清洗

In [4]:
tags_tmp = []
for tag in tags:
    if tag == "":
        tags_tmp.append("0")
    else:
        tags_tmp.append(tag)

tags = tags_tmp

n_tags = len(np.unique(tags))
labels = tags
print n_tags,np.unique(tags)

5 ['-1' '-2' '0' '1' '2']


4.lda模型，转换成sparse matrix，并用scale标准化

In [41]:
tfidf = models.TfidfModel(corpus)
tfidf_corpus = tfidf[corpus]

In [38]:
#load lda模型
lda = models.LdaMulticore.load("result/lda_model_caicai_tfidf_50")

In [3]:
#lda = models.LdaMulticore(tfidf_corpus, num_topics=150, passes=50, iterations=50, id2word=dictionary)
#lda.save("result/lda_model_jinjin_tfidf_150")

In [ ]:
#lda = models.LdaMulticore(corpus, num_topics=150, passes=50, iterations=50, id2word=dictionary)
#lda.save("result/lda_model_jinjin_150")

In [42]:
#aa = lda[corpus]
aa = lda[tfidf_corpus]
s_l = []
for ll in aa:
    s_l.append(ll)

In [43]:
s_m = ca.list2SP(s_l)
print s_m.col
print s_m.row

data = scale(s_m, with_mean=False)

[16 34 40 ..., 34 40 48]
[  0   0   0 ..., 539 539 539]


5.聚类，尝试k-means++/random/pca降维,比较效果

In [44]:
#设置参数，聚类个数及初始点个数
n_clusters = 10
n_init = 10

In [28]:
#k-means++
kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=n_init)
kmeans.fit(data)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=10, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [ ]:
#random
kmeans = KMeans(init='random', n_clusters=n_clusters, n_init=n_init)
kmeans.fit(data)

In [45]:
#pca降维+"k-means++"
reduced_data = PCA(n_components=5).fit_transform(data.toarray())
kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=n_init)
kmeans.fit(reduced_data)

#pca降维
#pca = PCA(n_components=5).fit(data.toarray())
#kmeans_pca = KMeans(init=pca.components_, n_clusters=10, n_init=10)
#kmeans_pca.fit(data)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=10, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

6.计算每个类的样本数以及到中心点最近的样本点

In [65]:
#clus_pred = kmeans.labels_
clus_pred = kmeans.predict(reduced_data)

#print set(clus_pred)
#print len(clus_pred)


k_s = {}
for i, p in enumerate(clus_pred):
    if p in k_s:
        k_s[p].append(i)
    else:
        k_s[p] = []
        k_s[p].append(i)
        
for k in k_s:
    print k,len(k_s[k])
 
#print k_s

0 129
1 131
2 73
3 35
4 29
5 31
6 30
7 37
8 22
9 23


In [99]:
centroids = kmeans.cluster_centers_
#print centroids

#print reduced_data[0:5],len(reduced_data)

def dist(x,y):
    return np.sqrt(np.sum((x-y)**2))

for i , cent in enumerate(centroids):
    ll = k_s[i]
    dis = []
    ind = []
    for index in ll:
        dis_index = dist(cent,reduced_data[index])
        dis.append(dis_index)
        ind.append(index)
    dis_min = np.min(dis)
    index_min = ll[dis.index(dis_min)]
    #dis_max = np.max(dis)
    #dis_mean = np.mean(dis)
    print i,dis_min,index_min#,dis_max,dis_mean
    #reduced_data[ll]
    #print i,cent,len(ll),reduced_data[ll]

cent = centroids[0]
data = reduced_data[289]
print dist(cent,data)

0 0.291108780961 289
1 0.442603693973 348
2 0.547483494503 359
3 0.37895093721 297
4 0.453194940295 409
5 0.330853212443 519
6 0.727555609374 304
7 0.227809048447 232
8 0.306354817742 472
9 0.30385854557 464
0.291108780961


In [82]:
a=[1,2,3]
b=[2,3,4]
#dis=np.linalg.norm(a,b)
#print dis
aa = np.array(a)
bb = np.array(b)
print a,type(a)
print aa,type(aa)

def dis(x,y):
    return np.sqrt(np.sum((x-y)**2))

dis = dis(aa,bb)
print dis


[1, 2, 3] <type 'list'>
[1 2 3] <type 'numpy.ndarray'>
1.73205080757
197.291662267


7.聚类结果输出到文件

In [60]:
f = open("news540")
news = []

for line in f:
    news.append(line)

f.close()

g = open("clus_result/k_means_result_50tfidf_10clus","w")
        
for k in k_s:
    k_l = k_s[k]
    for ll in k_l:
        g.write(("%s||%s\n") % (news[ll], k))
    
    g.write(("----End of the %d cluster---------------------------------------------\n") % k)
    g.write(("----------------------------------------------------------------------\n") % k)

g.flush()
g.close() 

8.附加：聚类效果比较：
前提，labels必须是事先打好的类别标签

In [50]:
sample_size = 100

def k_means_model(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))

In [52]:
k_means_model(KMeans(init='k-means++', n_clusters=n_clusters, n_init=n_init),
              name="k-means++", data=data)

k_means_model(KMeans(init='random', n_clusters=n_clusters, n_init=n_init),
              name="random", data=data)

pca = PCA(n_components=n_tags).fit(data.toarray())
k_means_model(KMeans(init=pca.components_, n_clusters=n_clusters, n_init=n_init),
              name="PCA-based",
              data=data)

k-means++   0.21s    22428   0.019   0.014   0.016   -0.001   -0.007    0.051
   random   0.06s    22446   0.027   0.031   0.029   -0.022   0.000    0.036
PCA-based   0.01s    24935   0.006   0.006   0.006   0.001   -0.006    0.034
